In [2]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [50]:
pd.set_option('display.max_columns', None)

In [3]:
!pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn"

geopandas==0.14.3
lightgbm @ file:///tmp/lightgbm/lightgbm-4.2.0-py3-none-manylinux_2_31_x86_64.whl#sha256=26ed21477c12bb26edc4d6d51336cd43d5a8f7daf55ebbe27b0faf50ce96db23
numpy @ file:///home/conda/feedstock_root/build_artifacts/numpy_1707225380409/work/dist/numpy-1.26.4-cp310-cp310-linux_x86_64.whl#sha256=51131fd8fc130cd168aecaf1bc0ea85f92e8ffebf211772ceb16ac2e7f10d7ca
pandas==2.1.4
pandas-datareader==0.10.0
pandas-profiling==3.6.6
pandas-summary==0.2.0
pandasql==0.7.3
scikit-learn==1.2.2
scikit-learn-intelex==2024.2.0
sklearn-pandas==2.2.0


In [186]:
train_df = pd.read_parquet("/kaggle/input/omegabankus/train_data.pqt")
test_df = pd.read_parquet("/kaggle/input/omegabankus/test_data.pqt")

In [187]:
def preproc_df(df):
    features_null = ['cnt_deb_g_oper_1m',
                     'cnt_days_deb_f_oper_1m',
                     'cnt_c_oper_1m',
                     'cnt_deb_d_oper_1m',
                     'cnt_cred_d_oper_1m',
                     'cnt_deb_e_oper_1m',
                     'cnt_cred_e_oper_1m',
                     'cnt_days_cred_e_oper_1m',
                     'cnt_deb_f_oper_1m',
                     'cnt_cred_f_oper_1m',
                     'cnt_a_oper_1m',
                     'cnt_days_cred_f_oper_1m',
                     'cnt_days_deb_g_oper_1m',
                     'cnt_cred_g_oper_1m',
                     'cnt_days_cred_g_oper_1m',
                     'cnt_deb_h_oper_1m',
                     'cnt_days_deb_h_oper_1m',
                     'cnt_cred_h_oper_1m',
                     'cnt_days_cred_h_oper_1m',
                     'cnt_b_oper_1m']
    
    print(len(features_null))
    mask = df[features_null].isnull().sum(axis=1) >= 20 # если больше-равно 17 нанов есть в этих фичах то скипус
    df_null = df[mask].drop(columns = features_null).reset_index()
    df = df[~mask].reset_index()
    
    return df_null, df

In [188]:
train_null, train = preproc_df(train_df)

20


In [52]:
train['end_cluster'].value_counts()

end_cluster
{α}          237061
{other}       41717
{α, η}        35892
{α, γ}        32268
{}            24281
{α, β}        10333
{α, θ}         5717
{α, ε}         4819
{α, δ}         3819
{α, ψ}         2360
{α, μ}         1868
{α, ε, η}      1540
{α, ε, θ}      1027
{α, λ}          730
{α, ε, ψ}       327
{λ}              46
{α, π}           14
Name: count, dtype: int64

In [53]:
train_null['end_cluster'].value_counts()

end_cluster
{}           98085
{α}          81609
{other}       5480
{α, η}        4918
{α, γ}        2203
{α, β}         870
{α, θ}         794
{α, ε}         631
{α, ψ}         569
{α, δ}         259
{α, μ}         255
{α, λ}         143
{α, ε, η}      134
{α, ε, θ}       98
{λ}             94
{α, ε, ψ}       34
{α, π}           5
Name: count, dtype: int64

In [48]:
train_null.shape[0] / train_df.shape[0]

0.32696833333333336

In [58]:
train

,index,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701,0.307935,0.302226,0.425993,1.135607,0.411205,0.864251,1.237550,-0.142187,0.286081,0.902845,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.125043,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.485936,2.773674,2.970940,2.472771,okved_30,segment_1,0.403604,0.613167,0.480547,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.485924,0.893049,0.560179,0.148894,0.873656,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.029795,0.223291,0.294665,1.251369,0.653444,0.387856,0.552757,0.629637,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.8709

In [63]:
train_df.describe()

,id,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,ogrn_days_end_month,ogrn_days_end_quarter,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m
count,600000.000000,5.344230e+05,5.344230e+05,5.344230e+05,5.344230e+05,5.685270e+05,5.685270e+05,5.686400e+05,2.644080e+05,2.644080e+05,5.685270e+05,4.511160e+05,4.511160e+05,4.511160e+05,6.000000e+05,403819.000000,6.000000e+05,403819.000000,6.000000e+05,403819.000000,6.000000e+05,403819.000000,6.000000e+05,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,403819.000000,403819.000000,6.000000e+05,434748.000000,6.000000e+05,434748.000000,6.000000e+05,434748.000000,6.000000e+05,434748.000000,6.000000e+05,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000,6.000000e+05,434748.000000,434748.000000
mean,99999.500000,5.291614e-18,-7.312532e-17,-3.959403e-17,-9.945043e-18,-6.117751e-17,1.003586e-17,-2.430370e-18,1.211971e-17,-3.759528e-17,4.538009e-17,3.076914e-17,-2.437433e-17,4.388166e-17,1.092341e-16,0.420404,1.162922e-17,0.437666,-2.406371e-17,0.551263,9.612459e-17,0.968775,-1.835569e-18,0.222822,1.097196e-17,0.893176,0.375884,-5.401013e-17,0.873523,0.231035,2.736774e-17,0.771360,0.243524,-2.232288e-18,0.154169,0.781854,-5.329071e-19,0.517408,0.941458,-9.651539e-18,0.224805,0.321208,3.631465e-17,0.653700,0.202554,-7.566392e-17,0.593454,0.134408,7.621755e-17,0.211545,1.736981e-17,0.517448,5.113539e-17,0.960474,-2.638482e-17,0.871546,-1.660302e-17,0.308652,-3.559819e-17,0.426119,1.005758,2.671937e-17,0.863885,0.847205,1.166711e-16,0.286103,0.917488,3.659295e-18,0.248516,0.042337,-4.121148e-18,0.461224,0.810852,7.401487e-19,0.946172,0.410920,3.422448e-18,0.551292,0.507565,4.041212e-17,0.255424,0.456226
std,57735.075031,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000002e+00,1.000002e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,0.013744,1.000001e+00,0.018238,1.000001e+00,0.003530,1.000001e+00,0.006968,1.000001e+00,0.007659,1.000001e+00,0.002576,0.237980,1.000001e+00,0.001993,0.231791,1.000001e+00,0.002094,0.108864,1.000001e+00,0.001925,0.030775,1.000001e+00,0.009505,0.163032

In [64]:
train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701,0.307935,0.302226,0.425993,1.135607,0.411205,0.864251,1.237550,-0.142187,0.286081,0.902845,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.125043,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.485936,2.773674,2.970940,2.472771,okved_30,segment_1,0.403604,0.613167,0.480547,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.485924,0.893049,0.560179,0.148894,0.873656,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.029795,0.223291,0.294665,1.251369,0.653444,0.387856,0.552757,0.629637,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701

In [146]:
ss = pd.read_csv('/kaggle/input/omegabankus/sample_submission.csv')

In [147]:
labels = {i:k for k,i in enumerate(list(ss)[1:])}
labels

{'{other}': 0,
 '{}': 1,
 '{α, β}': 2,
 '{α, γ}': 3,
 '{α, δ}': 4,
 '{α, ε, η}': 5,
 '{α, ε, θ}': 6,
 '{α, ε, ψ}': 7,
 '{α, ε}': 8,
 '{α, η}': 9,
 '{α, θ}': 10,
 '{α, λ}': 11,
 '{α, μ}': 12,
 '{α, π}': 13,
 '{α, ψ}': 14,
 '{α}': 15,
 '{λ}': 16}

In [148]:
train = train_null # COMMENT IT !!!!!!!!!!!!!!!!!!!!!!!!!!!!
train.head(3)

,index,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,sum_b_oper_1m,sum_c_oper_1m,sum_deb_d_oper_1m,sum_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,sum_deb_f_oper_1m,sum_cred_f_oper_1m,sum_deb_g_oper_1m,sum_cred_g_oper_1m,sum_deb_h_oper_1m,sum_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,9,3,month_1,-0.156643,-0.204861,-0.125660,-0.156179,channel_code_14,city_21,city_type_0,index_city_code_46,-1.185321,-0.367365,ogrn_month_4,ogrn_year_10,-0.424966,-0.435430,-0.338836,-0.432350,okved_10,segment_3,-0.273969,-0.268832,-0.242793,-0.188835,-0.030225,-0.103359,-0.128513,-0.063678,-0.142792,NaN,-0.143862,-0.173564,-0.017983,-0.041418,-0.029795,-0.158938,-0.171825,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,-0.152556,0.425738,0.761981,-0.156528,0.86376,0.644144,-0.210192,0.286061,0.869878,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.37454,{α},{α}
1,11,3,month_3,-0.156522,-0.204718,-0.125759,-0.156058,channel_code_14,city_21,city_type_0,index_city_code_46,-1.185321,-0.367365,ogrn_month_4,ogrn_year_10,-0.391211,-0.398217,-0.300456,-0.397558,okved_10,segment_3,-0.273969,-0.294447,-0.273303,-0.188835,-0.030225,-0.103359,-0.128513,-0.063678,-0.142792,NaN,-0.143862,-0.173564,-0.017983,-0.041418,-0.029795,-0.158938,-0.171825,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,-0.152954,0.425725,0.740003,-0.156528,0.86376,0.644144,-0.213518,0.286054,0.858889,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.37454,{α},"{α, γ}"
2,15,5,month_1,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,-0.188835,-0.030225,-0.103359,-0.128513,-0.063678,-0.142792,NaN,-0.143862,-0.173564,-0.017983,-0.041418,-0.029795,-0.158938,-0.171825,-0.207082,NaN,-0.029964,NaN,-0.119654,NaN,-0.143207,NaN,-0.084701,NaN,-0.152970,NaN,NaN,-0.156528,NaN,NaN,-0.213518,NaN,NaN,-0.025646,NaN,NaN,-0.040649,NaN,NaN,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{α}


In [192]:
def make_features(df):
    df['cnt_days_deb_full_oper_1m'] = np.sum(df[[x for x in list(df) if 'cnt_days_deb' in x and '1m' in x]],axis =1)
    df['cnt_days_deb_full_oper_2m'] = np.sum(df[[x for x in list(df) if 'cnt_days_deb' in x and '2m' in x]],axis =1)
    df['cnt_days_deb_full_oper_3m'] = np.sum(df[[x for x in list(df) if 'cnt_days_deb' in x and '3m' in x]],axis =1)
    
    
    for _ in ['a','b','c']:
        for __ in ['1','3']:
            try:
                df[f'avg_{_}_oper_{__}m'] = df[f'sum_{_}_oper_{__}m']/df[f'cnt_{_}_oper_{__}m']
            except:
                print(_,__,'err')
                ...

    for _ in ['d','e','f','g','h']:
        for __ in ['1','3']:
            for ___ in ['cred','deb']:
                try:
                    df[f'avg_{___}_{_}_oper_{__}m'] = df[f'sum_{___}_{_}_oper_{__}m']/df[f'cnt_{___}_{_}_oper_{__}m']
                except:
                    ...

    df['cluster_changes'] = df.groupby('id')['start_cluster'].transform('nunique')
    
    for i in range(1,6):
        df[f'noise_{i}'] = np.random.normal(0,0.6/i, df.shape[0])
    
    return df


In [25]:
good_features_null = ['ogrn_days_end_month', 'sum_of_paym_2m', 'sum_deb_h_oper_3m',
       'avg_cred_e_oper_3m', 'sum_deb_e_oper_3m',
       'cnt_days_deb_h_oper_3m', 'cnt_deb_e_oper_3m', 'avg_deb_d_oper_3m',
       'cnt_deb_g_oper_3m', 'sum_b_oper_3m', 'balance_amt_day_avg',
       'cluster_changes', 'channel_code', 'cnt_deb_h_oper_3m', 'okved',
       'cnt_b_oper_3m', 'sum_deb_d_oper_3m', 'max_founderpres',
       'avg_deb_f_oper_3m', 'avg_a_oper_3m', 'start_cluster',
       'ogrn_exist_months', 'ft_registration_date', 'cnt_a_oper_3m',
       'sum_deb_g_oper_3m', 'cnt_days_deb_e_oper_1m',
       'avg_cred_h_oper_3m', 'ogrn_month', 'cnt_days_cred_h_oper_3m',
       'sum_of_paym_1y', 'avg_deb_g_oper_3m', 'ogrn_year',
       'cnt_days_deb_e_oper_3m', 'cnt_days_cred_e_oper_3m', 'city',
       'min_founderpres', 'balance_amt_max', 'segment',
       'cnt_days_deb_f_oper_3m', 'avg_deb_h_oper_3m', 'cnt_deb_d_oper_3m',
       'balance_amt_avg', 'balance_amt_min', 'avg_deb_e_oper_3m',
       'sum_cred_e_oper_1m', 'cnt_cred_h_oper_3m', 'sum_cred_e_oper_3m',
       'cnt_days_deb_g_oper_3m', 'cnt_days_deb_full_oper_1m',
       'sum_cred_h_oper_3m', 'ogrn_days_end_quarter', 'sum_of_paym_6m',
       'sum_deb_e_oper_1m', 'cnt_days_deb_full_oper_3m']

In [367]:
good_features_not_null = ['start_cluster', 'okved', 'channel_code', 'city', 'ogrn_year',
                 'cluster_changes', 'max_founderpres', 'ogrn_month', 'balance_amt_min',
                 'min_founderpres', 'cnt_days_cred_e_oper_3m', 'cnt_days_cred_e_oper_1m',
                 'balance_amt_avg', 'balance_amt_day_avg', 'balance_amt_max', 'sum_of_paym_1y',
                 'segment', 'sum_of_paym_6m', 'ogrn_exist_months', 'ft_registration_date',
                 'cnt_days_deb_e_oper_3m', 'avg_cred_h_oper_3m', 'avg_cred_e_oper_3m',
                 'avg_deb_h_oper_3m', 'avg_cred_e_oper_1m', 'sum_of_paym_2m', 'ogrn_days_end_month',
                 'cnt_days_cred_h_oper_3m', 'cnt_days_deb_full_oper_3m', 'ogrn_days_end_quarter',
                 'avg_deb_e_oper_3m', 'cnt_days_deb_full_oper_1m', 'cnt_days_deb_e_oper_1m',
                 'avg_deb_e_oper_1m', 'sum_cred_e_oper_1m', 'cnt_days_deb_g_oper_3m', 
                 'sum_cred_h_oper_3m', 'cnt_days_deb_f_oper_3m', 'cnt_days_deb_h_oper_3m', 
                 'avg_deb_d_oper_3m', 'sum_deb_h_oper_3m', 'cnt_cred_h_oper_3m', 'avg_cred_h_oper_1m',
                 'avg_deb_f_oper_3m', 'avg_deb_h_oper_1m', 'cnt_days_deb_g_oper_1m',
                 'cnt_b_oper_3m', 'sum_cred_e_oper_3m', 'cnt_deb_e_oper_3m', 'cnt_deb_g_oper_3m', 
                 'avg_deb_g_oper_3m', 'cnt_deb_h_oper_1m', 'sum_deb_g_oper_3m',
                 'sum_deb_d_oper_3m', 'avg_deb_f_oper_1m', 'sum_deb_e_oper_3m', 'avg_a_oper_3m',
                 'cnt_deb_d_oper_3m', 'cnt_deb_h_oper_3m', 'cnt_cred_h_oper_1m', 'sum_b_oper_3m', 
                 'avg_deb_d_oper_1m', 'sum_deb_e_oper_1m', 'cnt_a_oper_3m']


In [158]:
good_features = [ft for ft in list(train)  if ft not in ['index', 'id', 'end_cluster']]

In [366]:
good_features_null = ['city', 'okved', 'channel_code', 'ogrn_year', 'ogrn_month',
       'index_city_code', 'start_cluster', 'balance_amt_min',
       'ogrn_days_end_month', 'ogrn_days_end_quarter',
       'ft_registration_date', 'balance_amt_avg', 'sum_of_paym_1y',
       'max_founderpres', 'min_founderpres', 'balance_amt_max',
       'sum_of_paym_6m', 'segment', 'date', 'cluster_changes',
       'balance_amt_day_avg', 'ogrn_exist_months', 'sum_of_paym_2m',
       'cnt_deb_e_oper_3m', 'avg_deb_e_oper_3m', 'city_type',
       'sum_cred_e_oper_3m', 'sum_deb_e_oper_3m', 'avg_cred_e_oper_3m',
       'cnt_cred_e_oper_3m', 'avg_deb_h_oper_3m', 'sum_deb_h_oper_3m']

In [308]:
def preproc_for_train(train, train_features = None, target = 'end_cluster', random_state_split = 42):
    if 'end_cluster' in list(train):
        train['end_cluster'] = train['end_cluster'].apply(lambda x: labels[x] if not str(x).isdigit() else x)
    train['start_cluster'] = train['start_cluster'].apply(lambda x: labels[x] if not str(x).isdigit() else x)
    cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"]
    train[cat_cols] = train[cat_cols].astype("category")
    
    train = make_features(train)
    
    if train_features is None:
        train_features = [ft for ft in list(train)  if ft not in ['index', 'id', 'end_cluster', target]]
        
    X = train[train_features]
    # X = X[X.columns[(X.isnull().sum() < 165252)]]
    y = train[target]


    # Кат фичи преведем в тип 'category'
    for col in cat_cols + ['date']:
        try:
            X[col] = X[col].astype('category')
        except:
            ...

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state_split, stratify=y)

    dtrain = xgb.DMatrix(X_train,
                         label=y_train,
                         nthread=-1,
                         enable_categorical=True,)

    dtest = xgb.DMatrix(X_test,
                        y_test,
                        nthread=-1,
                        enable_categorical=True)
    
    return X_train, X_test, y_train, y_test, dtrain, dtest

In [163]:
cluster_weights = pd.read_excel("/kaggle/input/omegabankus/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [342]:
def weighted_roc_auc_metric(predt: np.ndarray, dtrain: xgb.DMatrix):
    try:
        y_true = dtrain.get_label()
    except:
        y_true = dtrain
    n_classes = 17
#     print(predt.shape,dtrain.num_row(), n_classes)
    
#     assert predt.shape == (dtrain.num_row(), n_classes)
    
    # Reshape the predictions to 2D array (n_samples, n_classes)
    y_pred = predt.reshape(-1, n_classes)
    
    # Convert labels to one-hot encoding
    y_true_one_hot = np.eye(n_classes)[y_true.astype(int)]
    
    # Load weights
    
    
    # Calculate weighted ROC AUC
    score = weighted_roc_auc(y_true_one_hot, y_pred)
    
    return 'Weighted_ROC_AUC', score

def weighted_roc_auc(y_true, y_pred, weights_dict = weights_dict, labels = labels):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
#     print(y_true.sum(axis=0), y_true.shape[0], len(np.unique(y_true)))
    classes_roc_auc = roc_auc_score(y_true, y_pred, multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [266]:
# import optuna

# def objective(trial):
#     param = {
#         "silent": 1,
#         "objective": "multi:softprob",
#         "num_class": 17,
#         'tree_method': 'hist',
#         'device': "cuda",
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         "max_depth": trial.suggest_int("max_depth", 1, 9),
#         "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
#         "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
#         "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
#         "subsample": trial.suggest_float("subsample", 0.1, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
#         "max_bin": trial.suggest_int("max_bin", 2, 256),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
#         "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-6, 1e6, log=True),
#     }

#     dtrain = xgb.DMatrix(X_train, label=y_train,enable_categorical = True)
#     dvalid = xgb.DMatrix(X_test, label=y_test,enable_categorical = True)

#     model = xgb.train(param, dtrain, evals=[(dvalid, "validation")], verbose_eval=False, early_stopping_rounds=10, custom_metric = weighted_roc_auc_metric)
#     preds = model.predict(dvalid)
    
    
#     return weighted_roc_auc_metric(preds,dvalid)[1]

# study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=100)

# print("Number of finished trials: ", len(study.trials))
# print("Best trial:")
# trial = study.best_trial

# print("  Value: ", trial.value)
# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [233]:
train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,cnt_days_deb_full_oper_1m,cnt_days_deb_full_oper_2m,cnt_days_deb_full_oper_3m,avg_a_oper_1m,avg_a_oper_3m,avg_b_oper_1m,avg_b_oper_3m,avg_c_oper_1m,avg_c_oper_3m,avg_cred_d_oper_1m,avg_deb_d_oper_1m,avg_cred_d_oper_3m,avg_deb_d_oper_3m,avg_cred_e_oper_1m,avg_deb_e_oper_1m,avg_cred_e_oper_3m,avg_deb_e_oper_3m,avg_cred_f_oper_1m,avg_deb_f_oper_1m,avg_cred_f_oper_3m,avg_deb_f_oper_3m,avg_cred_g_oper_1m,avg_deb_g_oper_1m,avg_cred_g_oper_3m,avg_deb_g_oper_3m,avg_cred_h_oper_1m,avg_deb_h_oper_1m,avg_cred_h_oper_3m,avg_deb_h_oper_3m,cluster_changes,noise_1,noise_2,noise_3,noise_4,noise_5
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,3,0,1.912607,0.0,3.596258,-0.452818,-0.993386,-0.069323,-0.057924,-0.187652,-0.124637,-0.286845,-0.132894,-0.275061,-0.164582,0.578880,0.444003,0.504466,0.836919,-0.116648,-0.162341,-0.103201,-0.531706,0.441548,-0.080589,0.196411,-0.088743,1.804507,1.611801,3.164040,1.621255,1,1.210158,0.059221,0.262158,0.168282,-0.127024
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.1277

In [467]:
xtr, xt, ytr, yt, dtrain, dtest = preproc_for_train(train_null, train_features = good_features_null, random_state_split=13631581)
xtr

a 1 err
b 1 err
c 1 err


/tmp/ipykernel_34/3407469841.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')
/tmp/ipykernel_34/3407469841.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


,city,okved,channel_code,ogrn_year,ogrn_month,index_city_code,start_cluster,balance_amt_min,ogrn_days_end_month,ogrn_days_end_quarter,ft_registration_date,balance_amt_avg,sum_of_paym_1y,max_founderpres,min_founderpres,balance_amt_max,sum_of_paym_6m,segment,date,cluster_changes,balance_amt_day_avg,ogrn_exist_months,sum_of_paym_2m,cnt_deb_e_oper_3m,avg_deb_e_oper_3m,city_type,sum_cred_e_oper_3m,sum_deb_e_oper_3m,avg_cred_e_oper_3m,cnt_cred_e_oper_3m,avg_deb_h_oper_3m,sum_deb_h_oper_3m
194536,city_0,okved_22,channel_code_8,ogrn_year_10,ogrn_month_2,index_city_code_13,15,-0.125175,0.672727,1.413615,-0.364894,-0.156338,0.814047,-0.369204,-0.270532,-0.204217,-0.028656,segment_0,month_3,1,-0.155873,-0.362766,-0.273969,0.425725,-0.359221,city_type_0,-0.156528,-0.152929,-0.181217,0.86376,-0.302782,-0.165588
45259,city_114,okved_4,channel_code_21,ogrn_year_4,ogrn_month_3,index_city_code_73,15,NaN,-1.533705,-1.683741,-1.037697,NaN,NaN,-1.110931,-1.035528,NaN,NaN,segment_3,month_3,2,NaN,-1.041208,NaN,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
128890,city_994,okved_5,channel_code_9,ogrn_year_3,ogrn_month_3,NaN,1,-0.125830,-1.533705,-1.683741,2.514542,-0.156712,NaN,NaN,NaN,-0.204913,NaN,segment_3,month_3,2,-0.156248,2.507563,NaN,NaN,NaN,city_type_572,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
185877,city_371,okved_36,channel_code_9,ogrn_year_20,ogrn_month_9,index_city_code_21,1,-0.125831,0.672727,-0.948119,0.507576,-0.156712,NaN,0.592644,0.721489,-0.204913,NaN,segment_2,month_1,1,-0.156248,0.507030,NaN,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
153016,city_92,okved_5,channel_code_8,ogrn_year_4,ogrn_month_0,NaN,15,-0.125929,-0.256297,-0.057629,-0.872357,-0.156751,-0.295721,NaN,NaN,-0.204942,-0.294633,segment_3,month_2,1,-0.156288,-0.867249,-0.273969,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54731,city_6,okved_5,channel_code_21,ogrn_year_11,ogrn_month_4,NaN,1,NaN,1.485623,0.523125,-0.830020,NaN,NaN,NaN,NaN,NaN,NaN,segment_3,month_1,1,NaN,-0.832457,NaN,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
170024,city_96,okved_5,channel_code_0,ogrn_year_9,ogrn_month_8,index_city_code_66,15,-0.124612,-0.488553,-1.335288,-0.039363,-0.155056,-0.295934,-0.010325,0.099606,-0.204087,-0.294633,segment_0,month_3,1,-0.154587,-0.049640,-0.273969,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
141290,city_169,okved_17,channel_code_18,ogrn_year_11,ogrn_month_0,index_city_code_104,15,-0.124339,-0.256297,-0.057629,-0.681271,-0.156111,0.209132,-0.717992,-0.630262,-0.204462,-0.294590,segment_3,month_1,1,-0.155646,-0.693289,-0.273969,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588
38996,city_88,okved_46,NaN,ogrn_year_11,ogrn_month_8,index_city_code_65,15,-0.123196,-1.533705,-1.683741,-0.687565,-0.155652,NaN,-0.724930,-0.637418,-0.204116,NaN,segment_3,month_2,1,-0.155185,-0.693289,NaN,NaN,NaN,city_type_0,-0.156528,-0.152970,NaN,NaN,NaN,-0.165588


In [471]:
params_null = {
    'objective': 'multi:softprob',
    'tree_method': 'hist', # 'gpu_hist' # Место для ускорения
    'seed': 42,
    'num_class': 17,
#     'device': "cuda",
    'lambda': 2.88371131448781e-05,
    'lambda': 9.712629501240593e-06,
    'alpha': 0.8265318431324873,
    'max_depth': 2,
    'eta': 0.6958185151533428,
    'gamma': 1.8118490715965515e-08,
    'grow_policy': 'lossguide',
    'subsample': 0.7969921133471942,
    'colsample_bytree': 0.4542013406317802,
    'max_bin': 233,
    'min_child_weight': 3,
    'scale_pos_weight': 0.002067703148790483
}

booster_null = xgb.train(params_null,
                    dtrain=dtrain,
                    num_boost_round=50,
                    evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
#                     early_stopping_rounds=10,
                    verbose_eval=5,
                   custom_metric = weighted_roc_auc_metric)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:19:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	dtrain-mlogloss:1.21920	dtrain-Weighted_ROC_AUC:0.66104	dtest-mlogloss:1.21839	dtest-Weighted_ROC_AUC:0.64245
[5]	dtrain-mlogloss:0.93947	dtrain-Weighted_ROC_AUC:0.79511	dtest-mlogloss:0.94488	dtest-Weighted_ROC_AUC:0.76663
[10]	dtrain-mlogloss:0.90199	dtrain-Weighted_ROC_AUC:0.84301	dtest-mlogloss:0.91478	dtest-Weighted_ROC_AUC:0.80255
[15]	dtrain-mlogloss:0.88839	dtrain-Weighted_ROC_AUC:0.86373	dtest-mlogloss:0.90557	dtest-Weighted_ROC_AUC:0.81961
[20]	dtrain-mlogloss:0.87931	dtrain-Weighted_ROC_AUC:0.87504	dtest-mlogloss:0.89996	dtest-Weighted_ROC_AUC:0.82796
[25]	dtrain-mlogloss:0.87280	dtrain-Weighted_ROC_AUC:0.88254	dtest-mlogloss:0.89582	dtest-Weighted_ROC_AUC:0.83372
[30]	dtrain-mlogloss:0.86735	dtrain-Weighted_ROC_AUC:0.88777	dtest-mlogloss:0.89309	dtest-Weighted_ROC_AUC:0.84020
[35]	dtrain-mlogloss:0.86259	dtrain-Weighted_ROC_AUC:0.89162	dtest-mlogloss:0.89102	dtest-Weighted_ROC_AUC:0.84251
[40]	dtrain-mlogloss:0.85885	dtrain-Weighted_ROC_AUC:0.89457	dtest-mlogloss:0.8894

In [184]:
models = [booster_null]
imps = [[x,y_] for x,y_ in zip(models[0].get_score().keys(),models[0].get_score().values())]
imps = sorted(imps, key = lambda x: -x[1])
imps
# good_features = [x[0] for x in imps if x[1] > 77 and 'noise' not in x[0]]
# np.array(good_features)

[['okved', 3894.0],
 ['channel_code', 3882.0],
 ['ogrn_month', 3446.0],
 ['ogrn_year', 3027.0],
 ['ogrn_days_end_month', 2710.0],
 ['ogrn_days_end_quarter', 2481.0],
 ['balance_amt_min', 2443.0],
 ['city', 2133.0],
 ['index_city_code', 1964.0],
 ['ft_registration_date', 1779.0],
 ['sum_of_paym_1y', 1668.0],
 ['balance_amt_avg', 1537.0],
 ['balance_amt_max', 1512.0],
 ['min_founderpres', 1488.0],
 ['max_founderpres', 1406.0],
 ['date', 1263.0],
 ['start_cluster', 1230.0],
 ['sum_of_paym_6m', 1133.0],
 ['ogrn_exist_months', 1016.0],
 ['balance_amt_day_avg', 937.0],
 ['segment', 770.0],
 ['cluster_changes', 676.0],
 ['avg_deb_e_oper_3m', 520.0],
 ['avg_cred_e_oper_3m', 471.0],
 ['sum_of_paym_2m', 404.0],
 ['cnt_deb_e_oper_3m', 375.0],
 ['city_type', 266.0],
 ['avg_deb_h_oper_3m', 157.0],
 ['sum_deb_e_oper_3m', 147.0],
 ['cnt_cred_e_oper_3m', 133.0],
 ['sum_cred_e_oper_3m', 80.0],
 ['sum_deb_h_oper_3m', 30.0]]

In [185]:
booster_null.save_model('null_model_last_dance.json')

In [472]:
xtr, xt, ytr, yt, dtrain, dtest = preproc_for_train(train, train_features = good_features_not_null, random_state_split=13631581)
xtr

/tmp/ipykernel_34/3407469841.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


,start_cluster,okved,channel_code,city,ogrn_year,cluster_changes,max_founderpres,ogrn_month,balance_amt_min,min_founderpres,cnt_days_cred_e_oper_3m,cnt_days_cred_e_oper_1m,balance_amt_avg,balance_amt_day_avg,balance_amt_max,sum_of_paym_1y,segment,sum_of_paym_6m,ogrn_exist_months,ft_registration_date,cnt_days_deb_e_oper_3m,avg_cred_h_oper_3m,avg_cred_e_oper_3m,avg_deb_h_oper_3m,avg_cred_e_oper_1m,sum_of_paym_2m,ogrn_days_end_month,cnt_days_cred_h_oper_3m,cnt_days_deb_full_oper_3m,ogrn_days_end_quarter,avg_deb_e_oper_3m,cnt_days_deb_full_oper_1m,cnt_days_deb_e_oper_1m,avg_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_days_deb_g_oper_3m,sum_cred_h_oper_3m,cnt_days_deb_f_oper_3m,cnt_days_deb_h_oper_3m,avg_deb_d_oper_3m,sum_deb_h_oper_3m,cnt_cred_h_oper_3m,avg_cred_h_oper_1m,avg_deb_f_oper_3m,avg_deb_h_oper_1m,cnt_days_deb_g_oper_1m,cnt_b_oper_3m,sum_cred_e_oper_3m,cnt_deb_e_oper_3m,cnt_deb_g_oper_3m,avg_deb_g_oper_3m,cnt_deb_h_oper_1m,sum_deb_g_oper_3m,sum_deb_d_oper_3m,avg_deb_f_oper_1m,sum_deb_e_oper_3m,avg_a_oper_3m,cnt_deb_d_oper_3m,cnt_deb_h_oper_3m,cnt_cred_h_oper_1m,sum_b_oper_3m,avg_deb_d_oper_1m,sum_deb_e_oper_1m,cnt_a_oper_3m
388824,15,okved_33,channel_code_8,city_664,ogrn_year_10,1,NaN,ogrn_month_9,-0.075825,NaN,0.819968,0.236181,-0.131394,-0.130852,-0.179168,-0.258985,segment_3,-0.262439,-0.327975,-0.330567,1.179563,-0.652201,-0.159919,-0.302782,-0.142346,-0.241686,0.208215,0.528386,11.139325,-1.102987,-0.320591,7.286207,0.624695,-0.139669,-0.124336,0.904331,-0.166667,1.221527,0.407687,-0.164582,-0.165588,0.255545,-0.235780,0.110059,-0.244379,1.074624,0.517298,-0.138141,0.425929,0.463182,-0.083823,0.650374,-0.038826,-0.143207,0.078722,-0.136549,-0.993386,0.870124,0.546889,0.593922,-0.029964,-0.132894,-0.124731,0.208461
109507,15,okved_5,channel_code_0,city_0,ogrn_year_15,1,NaN,ogrn_month_3,-0.125995,NaN,0.699089,0.074890,-0.155907,-0.155441,-0.203060,-0.245700,segment_3,-0.279674,0.019944,0.022998,0.915827,-0.801529,-0.171063,-0.270777,-0.164708,-0.273969,1.717879,0.374540,8.766065,-0.599666,-0.338358,3.899110,0.173082,-0.158975,-0.143862,0.728507,-0.201123,0.858889,0.418798,0.058278,-0.148240,0.250924,-0.292154,-0.746426,-0.244379,0.848818,0.517298,-0.147760,0.425803,0.458055,-0.088743,0.650374,-0.040649,0.050809,-0.225022,-0.144074,-0.993386,0.871841,0.547462,0.588131,-0.029964,-0.132894,-0.141947,0.208461
167655,9,okved_5,channel_code_11,city_49,ogrn_year_6,1,1.187413,ogrn_month_4,-0.125849,-0.904776,1.347440,0.720052,-0.067814,-0.067075,0.183669,1.664065,segment_1,1.151101,1.046304,1.047077,1.388354,-0.535020,1.128408,2.568114,1.022072,0.781969,0.672727,0.517397,13.335295,0.252106,2.251999,8.253949,0.689212,0.952171,0.892939,1.157079,-0.136413,0.880867,1.018798,1.837310,1.530690,0.254968,-0.261900,-0.387464,2.401564,1.203656,0.517298,0.975048,0.427417,0.482353,0.947624,0.686593,0.457089,1.624709,-0.225022,0.962542,-0.993386,0.884287,0.596037,0.594888,-0.029964,1.139949,0.851290,0.208461
1597,9,okved_30,channel_code_14,city_114,ogrn_year_9,1,NaN,ogrn_month_4,-0.123777,NaN,1.182605,0.558761,-0.135656,-0.135127,-0.139830,-0.202989,segment_3,-0.198382,-0.206203,-0.210424,0.959783,-0.729363,-0.134917,-0.302782,-0.097948,-0.190523,-0.024041,0.429485,8.864599,0.019805,-0.269276,4.286207,0.302115,-0.098504,-0.085562,0.728507,-0.184068,0.858889,0.407687,-0.164582,-0.165588,0.252368,-0.242940,-0.746426,-0.244379,0.848818,0.517298,-0.116559,0.425861,0.458055,-0.088743,0.650374,-0.040649,-0.143207,-0.225022,-0.114674,7.863531,0.870124,0.546889,0.591027,-0.029964,-0.132894,-0.087960,0.233539
104479,3,okved_5,channel_code_12,city_14,ogrn_year_11,1,-0.738175,ogrn_month_11,-0.125990,-0.651078,0.787001,0.171664,-0.146018,-0.145521,-0.168069,-0.238232,segment_3,-0.236629,-0.710685,-0.699579,1.080662,-0.767552,-0.128939,-0.245685,-0.136024,-0.228828,-0.140169,0.385529,9.792072,-0.057629,-0.253324,5.253949,0.398889,-0.113549,-0.118811,0.783452,-0.192819,0.858889,0.541020,-0.164582,-0.135137,0.251212,-0.292154,-0.746426,-0.205709,0.913334,0.517

In [473]:
params = {
    'objective': 'multi:softprob',
    'tree_method': 'hist', # 'gpu_hist' # Место для ускорения
    'seed': 42,
    'num_class': 17,
    'device': "cuda",
    'lambda': 0.023010805853017707,
    'alpha': 3.70464222507275e-07,
    'max_depth': 4,
    'eta': 0.5692720104854774,
    'gamma': 6.951027682859445e-05,
    'grow_policy': 'depthwise',
    'subsample': 0.6031824510422559,
    'colsample_bytree': 0.9522306157039789,
    'max_bin': 39,
    'min_child_weight': 5,
    'scale_pos_weight': 11978.32048081804
}

booster = xgb.train(params,
                    dtrain=dtrain,
                    num_boost_round=50,
                    evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
#                     early_stopping_rounds=10,
                    verbose_eval=5,
                   custom_metric = weighted_roc_auc_metric)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:20:17] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	dtrain-mlogloss:1.06283	dtrain-Weighted_ROC_AUC:0.81281	dtest-mlogloss:1.06622	dtest-Weighted_ROC_AUC:0.81171
[5]	dtrain-mlogloss:0.83074	dtrain-Weighted_ROC_AUC:0.89658	dtest-mlogloss:0.84546	dtest-Weighted_ROC_AUC:0.88670
[10]	dtrain-mlogloss:0.78938	dtrain-Weighted_ROC_AUC:0.92369	dtest-mlogloss:0.81356	dtest-Weighted_ROC_AUC:0.90882
[15]	dtrain-mlogloss:0.76953	dtrain-Weighted_ROC_AUC:0.93799	dtest-mlogloss:0.80278	dtest-Weighted_ROC_AUC:0.91596
[20]	dtrain-mlogloss:0.75313	dtrain-Weighted_ROC_AUC:0.94437	dtest-mlogloss:0.79635	dtest-Weighted_ROC_AUC:0.92031
[25]	dtrain-mlogloss:0.73960	dtrain-Weighted_ROC_AUC:0.94830	dtest-mlogloss:0.79095	dtest-Weighted_ROC_AUC:0.92104
[30]	dtrain-mlogloss:0.72796	dtrain-Weighted_ROC_AUC:0.95176	dtest-mlogloss:0.78632	dtest-Weighted_ROC_AUC:0.92517
[35]	dtrain-mlogloss:0.71635	dtrain-Weighted_ROC_AUC:0.95479	dtest-mlogloss:0.78087	dtest-Weighted_ROC_AUC:0.92768
[40]	dtrain-mlogloss:0.70582	dtrain-Weighted_ROC_AUC:0.95738	dtest-mlogloss:0.7769

In [77]:
models = [booster_null, booster]
imps = [[x,y_] for x,y_ in zip(models[1].get_score().keys(),models[1].get_score().values())]
imps = sorted(imps, key = lambda x: -x[1])
# good_features = [x[0] for x in imps if x[1] > 77]
imps

[['channel_code', 4992.0],
 ['ogrn_month', 4763.0],
 ['okved', 4709.0],
 ['ogrn_year', 4242.0],
 ['city', 3509.0],
 ['start_cluster', 1970.0],
 ['ogrn_days_end_month', 1872.0],
 ['ogrn_days_end_quarter', 1631.0],
 ['sum_of_paym_1y', 1430.0],
 ['balance_amt_min', 1321.0],
 ['sum_of_paym_6m', 1069.0],
 ['segment', 942.0],
 ['balance_amt_max', 890.0],
 ['cnt_days_cred_e_oper_3m', 871.0],
 ['sum_of_paym_2m', 862.0],
 ['avg_deb_f_oper_3m', 839.0],
 ['ogrn_exist_months', 795.0],
 ['ft_registration_date', 790.0],
 ['cnt_deb_e_oper_3m', 786.0],
 ['balance_amt_avg', 759.0],
 ['cnt_days_deb_f_oper_3m', 759.0],
 ['cnt_days_cred_e_oper_1m', 723.0],
 ['cnt_days_deb_e_oper_3m', 685.0],
 ['balance_amt_day_avg', 677.0],
 ['cnt_days_deb_full_oper_3m', 602.0],
 ['avg_deb_f_oper_1m', 567.0],
 ['avg_deb_g_oper_3m', 547.0],
 ['cnt_days_deb_g_oper_3m', 525.0],
 ['avg_deb_h_oper_3m', 518.0],
 ['cnt_days_deb_full_oper_1m', 518.0],
 ['cnt_deb_g_oper_3m', 508.0],
 ['cnt_deb_d_oper_3m', 501.0],
 ['cnt_deb_h_oper

In [198]:
booster.save_model('not_null_model_last_dance.json')

In [199]:
def get_predict(model, dataset):
    p1 = xgb.DMatrix(dataset[model.feature_names],
                     nthread=-1,
                     enable_categorical=True)
    pred = model.predict(p1)
    test_pred_proba_df = pd.DataFrame(pred, columns=labels.keys())
    test_pred_proba_df['index'] = dataset['index']
    return test_pred_proba_df

def concat_data(df1, df2):
    return pd.concat([df1, df2], axis=0).sort_values(by='index').drop('index', axis=1)

In [474]:
test_df = make_features(test_df)
test_df['start_cluster'] = test_df['start_cluster'].apply(lambda x: labels[x] if not str(x).isdigit() else x)
test_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt_days_deb_full_oper_1m,cnt_days_deb_full_oper_2m,cnt_days_deb_full_oper_3m,avg_a_oper_1m,avg_a_oper_3m,avg_b_oper_1m,avg_b_oper_3m,avg_c_oper_1m,avg_c_oper_3m,avg_cred_d_oper_1m,avg_deb_d_oper_1m,avg_cred_d_oper_3m,avg_deb_d_oper_3m,avg_cred_e_oper_1m,avg_deb_e_oper_1m,avg_cred_e_oper_3m,avg_deb_e_oper_3m,avg_cred_f_oper_1m,avg_deb_f_oper_1m,avg_cred_f_oper_3m,avg_deb_f_oper_3m,avg_cred_g_oper_1m,avg_deb_g_oper_1m,avg_cred_g_oper_3m,avg_deb_g_oper_3m,avg_cred_h_oper_1m,avg_deb_h_oper_1m,avg_cred_h_oper_3m,avg_deb_h_oper_3m,cluster_changes,noise_1,noise_2,noise_3,noise_4,noise_5
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,NaN,-1.533705,-1.683741,ogrn_month_8,ogrn_year_11,-0.653810,NaN,NaN,-0.658497,okved_0,segment_3,0.416833,0.332409,0.676573,0.532156,0.421695,-0.030225,0.435995,1.123507,0.552726,-0.128513,0.96703,-0.063678,0.221993,0.278159,0.893289,0.818244,0.266375,0.873550,0.526503,4.591710,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.034588,0.515230,0.913334,0.078352,0.226247,0.326923,-0.144097,0.652523,0.291082,0.413722,0.605505,0.500338,0.957934,0.214730,-0.029964,0.517298,1.660281,0.963300,-0.143207,0.870124,-0.055135,0.309193,0.175471,0.426448,1.366376,0.143413,0.863866,0.951836,3.136945,0.286081,0.902845,-0.025646,0.24851,0.038562,0.026364,0.460730,0.849386,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,15,11.337227,0.0,18.298142,1.261947,4.461107,-0.069323,-0.057924,2.032664,1.723534,-0.286845,-0.132894,-0.178320,-0.164582,0.304934,0.311388,0.166013,0.411470,-0.116648,5.952824,-0.103201,10.965223,0.346310,-0.067130,0.011576,0.057222,0.683268,-0.220830,0.123376,-0.280473,1,-0.037645,0.222145,-0.088705,0.144937,-0.030417
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,NaN,-1.533705,-1.683741,ogrn_month_8,ogrn_year_11,-0.636647,NaN,NaN,-0.641101,okved_0,segment_3,0.433195,0.284317,0.688449,-0.069498,0.421695,-0.030225,0.435995,4.983369,0.556064,-0.128513,0.96703,0.089395,0.223973,0.119735,0.893536,0.850502,0.131938,0.873598,0.526503,1.399146,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.010645,0.514586,0.881076,-0.029795,0.223291,0.294665,-0.145537,0.652216,0.194308,0.363179,0.612262,0.532596,1.002756,0.216298,-0.029964,0.517298,3.022775,0.963687,-0.143207,0.870124,-0.008083,0.309193,0.195415,0.426482,1.

In [475]:
for col in cat_cols + ['date']:
    try:
        test_df[col] = test_df[col].astype('category')
    except:
        ...

In [476]:
test_null_5, test_5 = preproc_df(test_df[test_df['date'] == 'month_5'])
test_5

20


,index,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt_days_deb_full_oper_1m,cnt_days_deb_full_oper_2m,cnt_days_deb_full_oper_3m,avg_a_oper_1m,avg_a_oper_3m,avg_b_oper_1m,avg_b_oper_3m,avg_c_oper_1m,avg_c_oper_3m,avg_cred_d_oper_1m,avg_deb_d_oper_1m,avg_cred_d_oper_3m,avg_deb_d_oper_3m,avg_cred_e_oper_1m,avg_deb_e_oper_1m,avg_cred_e_oper_3m,avg_deb_e_oper_3m,avg_cred_f_oper_1m,avg_deb_f_oper_1m,avg_cred_f_oper_3m,avg_deb_f_oper_3m,avg_cred_g_oper_1m,avg_deb_g_oper_1m,avg_cred_g_oper_3m,avg_deb_g_oper_3m,avg_cred_h_oper_1m,avg_deb_h_oper_1m,avg_cred_h_oper_3m,avg_deb_h_oper_3m,cluster_changes,noise_1,noise_2,noise_3,noise_4,noise_5
0,1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,NaN,-1.533705,-1.683741,ogrn_month_8,ogrn_year_11,-0.636647,NaN,NaN,-0.641101,okved_0,segment_3,0.433195,0.284317,0.688449,-0.069498,0.421695,-0.030225,0.435995,4.983369,0.556064,-0.128513,0.967030,0.089395,0.223973,0.119735,0.893536,0.850502,0.131938,0.873598,0.526503,1.399146,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.010645,0.514586,0.881076,-0.029795,0.223291,0.294665,-0.145537,0.652216,0.194308,0.363179,0.612262,0.532596,1.002756,0.216298,-0.029964,0.517298,3.022775,0.963687,-0.143207,0.870124,-0.008083,0.309193,0.195415,0.426482,1.410332,0.189248,0.863929,1.061726,3.136964,0.286088,0.913834,-0.025646,0.24851,0.038562,0.003038,0.459615,0.805430,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,15,10.853356,0.0,18.408642,-0.164805,4.636003,-0.069323,-0.057924,8.961860,3.136678,0.399132,-0.132894,-0.026143,-0.164582,0.151028,0.134001,0.219055,0.458201,-0.116648,1.813893,-0.103201,10.965027,-0.133434,-0.020686,0.007207,0.006610,0.593176,-0.223142,0.900105,-0.273910,1,-0.480488,0.200756,0.308414,-0.055273,0.123333
1,7,200002,month_5,0.572242,1.502779,-0.125995,0.574963,channel_code_12,city_14,city_type_0,index_city_code_78,-1.069193,-1.528873,ogrn_month_6,ogrn_year_11,-0.739055,-0.781695,-0.695963,-0.745477,okved_63,segment_3,3.870911,1.620822,0.970531,21.770544,0.505807,-0.030225,0.435995,3.847296,0.552726,5.315392,0.987058,-0.063678,0.221993,2.786193,0.894125,0.882760,2.792534,0.873497,0.397471,0.231784,0.771379,0.309302,-0.017983,0.154163,0.777702,0.251115,0.521660,1.171398,0.037447,0.225262,0.359181,5.075587,0.673701,0.678179,7.251601,0.597783,0.339047,11.289349,0.250780,-0.029964,0.517298,2.166535,0.961121,9.210451,0.885575,-0.

In [477]:
df_5 = pd.concat([train_df.drop('end_cluster', axis=1), test_df[test_df.date != 'month_6']])
df_5.head(3)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt_days_deb_full_oper_1m,cnt_days_deb_full_oper_2m,cnt_days_deb_full_oper_3m,avg_a_oper_1m,avg_a_oper_3m,avg_b_oper_1m,avg_b_oper_3m,avg_c_oper_1m,avg_c_oper_3m,avg_cred_d_oper_1m,avg_deb_d_oper_1m,avg_cred_d_oper_3m,avg_deb_d_oper_3m,avg_cred_e_oper_1m,avg_deb_e_oper_1m,avg_cred_e_oper_3m,avg_deb_e_oper_3m,avg_cred_f_oper_1m,avg_deb_f_oper_1m,avg_cred_f_oper_3m,avg_deb_f_oper_3m,avg_cred_g_oper_1m,avg_deb_g_oper_1m,avg_cred_g_oper_3m,avg_deb_g_oper_3m,avg_cred_h_oper_1m,avg_deb_h_oper_1m,avg_cred_h_oper_3m,avg_deb_h_oper_3m,cluster_changes,noise_1,noise_2,noise_3,noise_4,noise_5
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,3,7.650427,0.0,14.385034,-0.452818,-0.993386,-0.069323,-0.057924,-0.187652,-0.124637,-0.286845,-0.132894,-0.275061,-0.164582,0.578880,0.444003,0.504466,0.836919,-0.116648,-0.162341,-0.103201,-0.531706,0.441548,-0.080589,0.196411,-0.088743,1.804507,1.611801,3.164040,1.621255,1,-0.078799,-0.098309,-0.093362,-0.016791,0.132185
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.87098

In [397]:
df_5['start_cluster'].value_counts()

start_cluster
15    491192
1     107072
9      57458
3      43129
0      42545
2      13498
4      11350
8       7464
10      6348
14      3876
12      1883
5       1620
6       1083
11       997
7        462
16       114
13        29
Name: count, dtype: int64

In [478]:
test_null_5, test_5 = preproc_df(df_5)

20


In [479]:
test_null_5 = pd.concat([test_null_5, test_5[test_5['start_cluster'] == 13].sample(2, random_state=32)[list(test_null_5)]])
test_null_5

,index,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,sum_b_oper_1m,sum_c_oper_1m,sum_deb_d_oper_1m,sum_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,sum_deb_f_oper_1m,sum_cred_f_oper_1m,sum_deb_g_oper_1m,sum_cred_g_oper_1m,sum_deb_h_oper_1m,sum_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt_days_deb_full_oper_1m,cnt_days_deb_full_oper_2m,cnt_days_deb_full_oper_3m,avg_a_oper_1m,avg_a_oper_3m,avg_b_oper_1m,avg_b_oper_3m,avg_c_oper_1m,avg_c_oper_3m,avg_cred_d_oper_1m,avg_deb_d_oper_1m,avg_cred_d_oper_3m,avg_deb_d_oper_3m,avg_cred_e_oper_1m,avg_deb_e_oper_1m,avg_cred_e_oper_3m,avg_deb_e_oper_3m,avg_cred_f_oper_1m,avg_deb_f_oper_1m,avg_cred_f_oper_3m,avg_deb_f_oper_3m,avg_cred_g_oper_1m,avg_deb_g_oper_1m,avg_cred_g_oper_3m,avg_deb_g_oper_3m,avg_cred_h_oper_1m,avg_deb_h_oper_1m,avg_cred_h_oper_3m,avg_deb_h_oper_3m,cluster_changes,noise_1,noise_2,noise_3,noise_4,noise_5
0,9,3,month_1,-0.156643,-0.204861,-0.125660,-0.156179,channel_code_14,city_21,city_type_0,index_city_code_46,-1.185321,-0.367365,ogrn_month_4,ogrn_year_10,-0.424966,-0.435430,-0.338836,-0.432350,okved_10,segment_3,-0.273969,-0.268832,-0.242793,-0.188835,-0.030225,-0.103359,-0.128513,-0.063678,-0.142792,NaN,-0.143862,-0.173564,-0.017983,-0.041418,-0.029795,-0.158938,-0.171825,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,-0.152556,0.425738,0.761981,-0.156528,0.863760,0.644144,-0.210192,0.286061,0.869878,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,15,0.000000,0.0,11.072213,NaN,-0.993386,NaN,-0.057924,NaN,-0.124637,NaN,NaN,-0.275061,-0.164582,NaN,NaN,-0.181217,-0.358333,NaN,NaN,-0.103201,-0.734781,NaN,NaN,-0.030264,-0.088743,NaN,NaN,-0.801529,-0.302782,1,0.018610,-0.200782,-0.152421,0.073579,0.120357
1,11,3,month_3,-0.156522,-0.204718,-0.125759,-0.156058,channel_code_14,city_21,city_type_0,index_city_code_46,-1.185321,-0.367365,ogrn_month_4,ogrn_year_10,-0.391211,-0.398217,-0.300456,-0.397558,okved_10,segment_3,-0.273969,-0.294447,-0.273303,-0.188835,-0.030225,-0.103359,-0.128513,-0.063678,-0.142792,NaN,-0.143862,-0.173564,-0.017983,-0.041418,-0.029795,-0.158938,-0.171825,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,-0.152954,0.425725,0.740003,-0.156528,0.863760,0.644144,-0.213518,0.286054,0.858889,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,15,0.000000,0.0,10.940345,NaN,-0.993386,NaN,-0.057924,NaN,-0.124637,NaN,NaN,-0.275061,-0.164582,NaN,NaN,-0.181217,-0.359278,NaN,NaN,-0.103201,-0.746426,NaN,NaN,-0.030264,-0.088743,NaN,NaN,-0.801529,-0.302782,1,0.874598,0.016980,-0.434352,-0.150875,0.087444
2,15,5,month_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.188835,-0.030225,-0.103359,-0.128513,-0.063678,-0.142792,NaN,-0.143862,-0.173564,-0.017983,-0.041418,-0.029795,-0.158938,-0.171825,-0.207082,NaN,-0.029964,NaN,-0.119654,NaN,-0.1

In [480]:
features_start_null = ['channel_code', 'okved', 'ogrn_month', 'ogrn_year',
       'index_city_code', 'balance_amt_min', 'city', 'balance_amt_max',
       'ogrn_days_end_quarter', 'date', 'balance_amt_avg',
       'ogrn_days_end_month', 'max_founderpres', 'min_founderpres',
       'ogrn_exist_months', 'sum_of_paym_1y', 'balance_amt_day_avg',
       'segment', 'ft_registration_date', 'cluster_changes',
       'sum_of_paym_6m', 'sum_of_paym_2m', 'avg_deb_e_oper_3m',
       'avg_cred_e_oper_3m', 'cnt_deb_e_oper_3m', 'city_type']

In [481]:
features_start = ['okved', 'index_city_code', 'channel_code', 'city', 'ogrn_year',
       'ogrn_month', 'balance_amt_day_avg', 'cnt_days_cred_e_oper_3m',
       'min_founderpres', 'cnt_deb_e_oper_3m', 'sum_of_paym_1y',
       'balance_amt_min', 'cnt_cred_e_oper_3m', 'balance_amt_avg',
       'sum_of_paym_2m', 'max_founderpres', 'balance_amt_max',
       'ogrn_days_end_quarter', 'ogrn_exist_months',
       'cnt_days_deb_full_oper_3m', 'sum_deb_e_oper_3m',
       'avg_deb_e_oper_3m', 'cnt_days_cred_e_oper_1m',
       'cnt_cred_e_oper_1m', 'avg_cred_h_oper_3m', 'cluster_changes',
       'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m',
       'cnt_deb_e_oper_1m', 'sum_deb_f_oper_3m', 'ogrn_days_end_month',
       'avg_cred_e_oper_3m', 'sum_deb_h_oper_3m', 'ft_registration_date',
       'cnt_deb_f_oper_3m', 'sum_of_paym_6m', 'avg_deb_g_oper_3m',
       'sum_cred_h_oper_3m', 'cnt_deb_h_oper_3m']

In [482]:
test_5 = pd.concat([test_5, test_5[test_5['start_cluster'] == 16].sample(1, random_state=32)])

In [483]:
test_5['start_cluster'].value_counts()

start_cluster
15    340236
9      55926
3      42422
0      41443
2      13328
4      11158
8       6750
10      5833
14      2935
12      1829
5       1601
1       1331
6       1056
11       812
7        430
13        28
16         3
Name: count, dtype: int64

In [484]:
xtr, xt, ytr, yt, dtrain, dtest = preproc_for_train(test_5, train_features = features_start, target = 'start_cluster', random_state_split=13631581)

/tmp/ipykernel_34/3407469841.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


In [441]:
yt.value_counts()

start_cluster
15    68047
9     11185
3      8484
0      8289
2      2666
4      2232
8      1350
10     1167
14      587
12      366
5       320
1       266
6       211
11      162
7        86
13        6
16        1
Name: count, dtype: int64

In [490]:
# import optuna
# X_train, X_test, y_train, y_test  = xtr, xt, ytr, yt
# def objective(trial):
#     param = {
#         "silent": 1,
#         "objective": "multi:softprob",
#         "num_class": 17,
#         'tree_method': 'hist',
#         'device': "cuda",
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         "max_depth": trial.suggest_int("max_depth", 1, 9),
#         "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
#         "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
#         "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
#         "subsample": trial.suggest_float("subsample", 0.1, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
#         "max_bin": trial.suggest_int("max_bin", 2, 256),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
#         "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-6, 1e6, log=True),
#     }

#     dtrain = xgb.DMatrix(X_train, label=y_train,enable_categorical = True)
#     dvalid = xgb.DMatrix(X_test, label=y_test,enable_categorical = True)

#     model = xgb.train(param, dtrain, evals=[(dvalid, "validation")], verbose_eval=False, early_stopping_rounds=10, custom_metric = weighted_roc_auc_metric)
#     preds = model.predict(dvalid)
    
    
#     return weighted_roc_auc_metric(preds,dvalid)[1]

# study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=100)

# print("Number of finished trials: ", len(study.trials))
# print("Best trial:")
# trial = study.best_trial

# print("  Value: ", trial.value)
# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [466]:
params = {
    'objective': 'multi:softprob',
    'tree_method': 'approx', # 'gpu_hist' # Место для ускорения
    'seed': 42,
    'num_class': 17,
    'device': "cuda",
    'lambda': 5.481416264599405e-08,
    'alpha': 0.0682917457895306,
    'max_depth': 4,
    'eta': 0.7344862388958335,
    'gamma': 0.024858174990175938,
    'grow_policy': 'lossguide',
    'subsample': 0.8274726583476334,
    'colsample_bytree': 0.8822570644328338,
    'max_bin': 100,
    'min_child_weight': 4,
    'scale_pos_weight': 0.0001220361736803952
}

booster_5 = xgb.train(params,
                    dtrain=dtrain,
                    num_boost_round=50,
                    evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
#                     early_stopping_rounds=10,
                    verbose_eval=5,
                   custom_metric = weighted_roc_auc_metric
                     )

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:13:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	dtrain-mlogloss:1.13002	dtrain-Weighted_ROC_AUC:0.73799	dtest-mlogloss:1.13321	dtest-Weighted_ROC_AUC:0.74154
[5]	dtrain-mlogloss:0.91399	dtrain-Weighted_ROC_AUC:0.85974	dtest-mlogloss:0.93308	dtest-Weighted_ROC_AUC:0.85080
[10]	dtrain-mlogloss:0.86041	dtrain-Weighted_ROC_AUC:0.90358	dtest-mlogloss:0.88940	dtest-Weighted_ROC_AUC:0.89172
[15]	dtrain-mlogloss:0.81037	dtrain-Weighted_ROC_AUC:0.92271	dtest-mlogloss:0.84943	dtest-Weighted_ROC_AUC:0.90463
[20]	dtrain-mlogloss:0.78379	dtrain-Weighted_ROC_AUC:0.93136	dtest-mlogloss:0.83227	dtest-Weighted_ROC_AUC:0.91065
[25]	dtrain-mlogloss:0.75968	dtrain-Weighted_ROC_AUC:0.93778	dtest-mlogloss:0.81716	dtest-Weighted_ROC_AUC:0.91518
[30]	dtrain-mlogloss:0.74040	dtrain-Weighted_ROC_AUC:0.94226	dtest-mlogloss:0.80618	dtest-Weighted_ROC_AUC:0.91800
[35]	dtrain-mlogloss:0.72330	dtrain-Weighted_ROC_AUC:0.94617	dtest-mlogloss:0.79627	dtest-Weighted_ROC_AUC:0.92139
[40]	dtrain-mlogloss:0.70824	dtrain-Weighted_ROC_AUC:0.94932	dtest-mlogloss:0.7884

In [438]:
models = [booster_5]
imps = [[x,y_] for x,y_ in zip(models[0].get_score().keys(),models[0].get_score().values())]
imps = sorted(imps, key = lambda x: -x[1])
good_features = [x[0] for x in imps if x[1] > 152.0 and 'noise' not in x[0]]
np.array(good_features)

array(['okved', 'index_city_code', 'channel_code', 'city', 'ogrn_year',
       'ogrn_month', 'balance_amt_day_avg', 'cnt_days_cred_e_oper_3m',
       'min_founderpres', 'cnt_deb_e_oper_3m', 'sum_of_paym_1y',
       'balance_amt_min', 'cnt_cred_e_oper_3m', 'balance_amt_avg',
       'sum_of_paym_2m', 'max_founderpres', 'balance_amt_max',
       'ogrn_days_end_quarter', 'ogrn_exist_months',
       'cnt_days_deb_full_oper_3m', 'sum_deb_e_oper_3m',
       'avg_deb_e_oper_3m', 'cnt_days_cred_e_oper_1m',
       'cnt_cred_e_oper_1m', 'avg_cred_h_oper_3m', 'cluster_changes',
       'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m',
       'cnt_deb_e_oper_1m', 'sum_deb_f_oper_3m', 'ogrn_days_end_month',
       'avg_cred_e_oper_3m', 'sum_deb_h_oper_3m', 'ft_registration_date',
       'cnt_deb_f_oper_3m', 'sum_of_paym_6m', 'avg_deb_g_oper_3m',
       'sum_cred_h_oper_3m', 'cnt_deb_h_oper_3m'], dtype='<U25')

In [444]:
booster_5.save_model('start_cl_model_not_null.json')

In [423]:
params_null = {
    'objective': 'multi:softprob',
    'tree_method': 'approx', # 'gpu_hist' # Место для ускорения
    'seed': 42,
    'num_class': 17,
    'device': "cuda",
    'lambda': 2.88371131448781e-05,
    'lambda': 9.712629501240593e-06,
    'alpha': 0.8265318431324873,
    'max_depth': 2,
    'eta': 0.6958185151533428,
    'gamma': 1.8118490715965515e-08,
    'grow_policy': 'lossguide',
    'subsample': 0.7969921133471942,
    'colsample_bytree': 0.4542013406317802,
    'max_bin': 233,
    'min_child_weight': 3,
    'scale_pos_weight': 0.002067703148790483
}

booster_5_null = xgb.train(params_null,
                    dtrain=dtrain,
                    num_boost_round=50,
                    evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
#                     early_stopping_rounds=10,
                    verbose_eval=5,
                   custom_metric = weighted_roc_auc_metric
                     )

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:06:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	dtrain-mlogloss:0.85931	dtrain-Weighted_ROC_AUC:0.50338	dtest-mlogloss:0.85717	dtest-Weighted_ROC_AUC:0.52206
[5]	dtrain-mlogloss:0.54847	dtrain-Weighted_ROC_AUC:0.76761	dtest-mlogloss:0.54374	dtest-Weighted_ROC_AUC:0.78584
[10]	dtrain-mlogloss:0.51199	dtrain-Weighted_ROC_AUC:0.88581	dtest-mlogloss:0.51214	dtest-Weighted_ROC_AUC:0.87013
[15]	dtrain-mlogloss:0.49834	dtrain-Weighted_ROC_AUC:0.91797	dtest-mlogloss:0.50465	dtest-Weighted_ROC_AUC:0.89220
[20]	dtrain-mlogloss:0.48988	dtrain-Weighted_ROC_AUC:0.92964	dtest-mlogloss:0.49990	dtest-Weighted_ROC_AUC:0.89380
[25]	dtrain-mlogloss:0.48382	dtrain-Weighted_ROC_AUC:0.93544	dtest-mlogloss:0.49676	dtest-Weighted_ROC_AUC:0.90020
[30]	dtrain-mlogloss:0.47796	dtrain-Weighted_ROC_AUC:0.94122	dtest-mlogloss:0.49382	dtest-Weighted_ROC_AUC:0.90125
[35]	dtrain-mlogloss:0.47254	dtrain-Weighted_ROC_AUC:0.94503	dtest-mlogloss:0.49070	dtest-Weighted_ROC_AUC:0.90179
[40]	dtrain-mlogloss:0.46837	dtrain-Weighted_ROC_AUC:0.94768	dtest-mlogloss:0.4883

In [424]:
booster_5_null.save_model('start_cl_model_null.json')

In [418]:
models = [booster_5]
imps = [[x,y_] for x,y_ in zip(models[0].get_score().keys(),models[0].get_score().values())]
imps = sorted(imps, key = lambda x: -x[1])
good_features = [x[0] for x in imps if x[1] > 225.0 and 'noise' not in x[0]]
np.array(good_features)

array(['channel_code', 'okved', 'ogrn_month', 'ogrn_year',
       'index_city_code', 'balance_amt_min', 'city', 'balance_amt_max',
       'ogrn_days_end_quarter', 'date', 'balance_amt_avg',
       'ogrn_days_end_month', 'max_founderpres', 'min_founderpres',
       'ogrn_exist_months', 'sum_of_paym_1y', 'balance_amt_day_avg',
       'segment', 'ft_registration_date', 'cluster_changes',
       'sum_of_paym_6m', 'sum_of_paym_2m', 'avg_deb_e_oper_3m',
       'avg_cred_e_oper_3m', 'cnt_deb_e_oper_3m', 'city_type'],
      dtype='<U21')

In [304]:
weighted_roc_auc_metric(booster_5.predict(dtest), np.array(yt))

[4.600e+01 3.411e+03 1.000e+00 2.300e+01 2.000e+00 0.000e+00 2.000e+00
 1.000e+00 2.000e+01 5.100e+01 1.200e+01 1.000e+01 3.000e+00 0.000e+00
 2.300e+01 3.841e+03 2.000e+00] 7448 2


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [209]:
# f1, f2 = get_predict(booster, test_5), get_predict(booster_null, test_null_5)
# final = concat_data(f1, f2)
# final

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.003813,0.000518,7.284118e-03,0.011247,1.936375e-03,7.619016e-05,0.000106,4.773866e-06,0.000824,0.002218,0.002577,0.000479,0.000253,1.761187e-06,0.000802,0.967857,0.000004
0,0.070080,0.903340,4.496881e-05,0.000492,2.065714e-06,1.539151e-05,0.000004,1.103175e-05,0.000017,0.000146,0.000005,0.002974,0.012826,2.358709e-07,0.000043,0.009073,0.000926
1,0.933327,0.000607,1.771392e-03,0.015209,1.259524e-02,2.807793e-05,0.001069,3.228477e-04,0.000586,0.003657,0.010938,0.000525,0.000650,4.212952e-06,0.000888,0.017808,0.000013
1,0.004036,0.967108,6.358408e-07,0.000016,8.459798e-07,6.402584e-07,0.000077,7.479674e-06,0.000526,0.005305,0.000061,0.000132,0.000053,7.369385e-07,0.000002,0.022425,0.000250
2,0.072381,0.163822,1.642122e-02,0.045759,3.416981e-03,4.035332e-03,0.001105,1.500156e-04,0.007861,0.106881,0.013031,0.001399,0.004411,1.113992e-06,0.002296,0.556682,0.000346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37236,0.827352,0.119043,1.289158e-05,0.000031,1.121863e-05,8.078342e-07,0.000010,6.345304e-06,0.000782,0.000022,0.000349,0.001499,0.000005,1.973803e-05,0.000126,0.011525,0.039204
62759,0.012842,0.092439,8.623647e-03,0.022139,3.933459e-03,2.065491e-06,0.000092,7.373821e-07,0.000722,0.002903,0.003622,0.000374,0.000471,2.424376e-05,0.000691,0.851105,0.000017
62760,0.018153,0.005391,5.644932e-03,0.030844,1.988449e-03,6.305466e-05,0.000017,9.240535e-05,0.010399,0.001915,0.003911,0.000022,0.000120,5.990545e-07,0.027852,0.893581,0.000003
37237,0.072381,0.163822,1.642122e-02,0.045759,3.416981e-03,4.035332e-03,0.001105,1.500156e-04,0.007861,0.106881,0.013031,0.001399,0.004411,1.113992e-06,0.002296,0.556682,0.000346


In [491]:
test_6 = test_df[test_df['date'] == 'month_6']
test_null_6, test_6 = preproc_df(test_6)
test_null_6['start_cluster'] = get_predict(booster_5_null, test_null_6).drop('index', axis=1).idxmax(axis=1).apply(lambda x: labels[x])
test_6['start_cluster'] = get_predict(booster_5, test_6).drop('index', axis=1).idxmax(axis=1).apply(lambda x: labels[x])
test_6['start_cluster']

20


0         0
1         8
2        15
3         8
4        15
         ..
71604    15
71605    15
71606    15
71607    15
71608    15
Name: start_cluster, Length: 71609, dtype: int64

In [363]:
# start_labels = get_predict(booster_5, test_6).drop('index', axis=1).idxmax(axis=1).apply(lambda x: labels[x])
# start_labels

0         0
1        15
2         0
3        15
4        15
         ..
99995    15
99996    15
99997    15
99998    15
99999    15
Length: 100000, dtype: int64

In [364]:
# test_6['start_cluster'] = list(start_labels)
# test_6.head(3)

/tmp/ipykernel_34/3226370442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_6['start_cluster'] = list(start_labels)


,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,cnt_days_deb_full_oper_1m,cnt_days_deb_full_oper_2m,cnt_days_deb_full_oper_3m,avg_a_oper_1m,avg_a_oper_3m,avg_b_oper_1m,avg_b_oper_3m,avg_c_oper_1m,avg_c_oper_3m,avg_cred_d_oper_1m,avg_deb_d_oper_1m,avg_cred_d_oper_3m,avg_deb_d_oper_3m,avg_cred_e_oper_1m,avg_deb_e_oper_1m,avg_cred_e_oper_3m,avg_deb_e_oper_3m,avg_cred_f_oper_1m,avg_deb_f_oper_1m,avg_cred_f_oper_3m,avg_deb_f_oper_3m,avg_cred_g_oper_1m,avg_deb_g_oper_1m,avg_cred_g_oper_3m,avg_deb_g_oper_3m,avg_cred_h_oper_1m,avg_deb_h_oper_1m,avg_cred_h_oper_3m,avg_deb_h_oper_3m,cluster_changes,noise_1,noise_2,noise_3,noise_4,noise_5,index
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,NaN,-1.533705,-1.683741,ogrn_month_8,ogrn_year_11,-0.618911,NaN,NaN,-0.623705,okved_0,segment_3,0.223961,0.285376,0.671862,2.698111,0.431041,-0.030225,0.435995,4.186482,0.557473,-0.128513,0.967030,-0.063678,0.221993,-0.002000,0.893674,0.753728,0.090933,0.873559,0.526503,-0.173564,0.771321,0.18027,-0.017983,0.154163,0.777702,0.000068,0.515230,0.913334,-0.029795,0.223291,0.294665,-0.150212,0.651909,0.194308,0.219106,0.605505,0.468080,1.182043,0.216298,-0.029964,0.517298,4.162265,0.965149,-0.143207,0.870124,-0.014715,0.308564,0.149694,0.426615,1.465277,0.185240,0.863974,1.171616,2.470867,0.286081,0.902845,-0.025646,0.24851,0.038562,-0.013602,0.459170,0.783452,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,0,4.083278,0.0,7.385191,6.259527,5.464893,-0.069323,-0.057924,7.509743,4.312563,-0.286845,-0.132894,-0.047689,-0.164582,0.104095,-0.002238,0.214405,0.350888,-0.116648,-0.225022,-0.103201,8.636941,-0.133434,0.000133,0.007207,-0.029624,0.361857,-0.230418,1.443216,-0.278087,1,0.399187,-0.360059,0.069788,-0.017896,-0.007017,2
5,200001,month_6,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,NaN,0.092087,1.220030,ogrn_month_10,ogrn_year_11,-0.628065,NaN,NaN,-0.623705,okved_6,segment_3,NaN,NaN,NaN,-0.188835,NaN,-0.030225,NaN,-0.103359,NaN,-0.128513,NaN,-0.063678,NaN,-0.142792,NaN,NaN,-0.143862,NaN,NaN,-0.173564,NaN,NaN,-0.017983,NaN,NaN,-0.041418,NaN,NaN,-0.029795,NaN,NaN,-0.158938,NaN,NaN,-0.171825,NaN,NaN,-0.207082,NaN,-0.029964,NaN,-0.119654,NaN,-0.143207,NaN,-0.084701,NaN,-0.152970,NaN,NaN,-0.156528,NaN,NaN,-0.213518,NaN,NaN,-0.025646,NaN,NaN,-0.040649,NaN,NaN,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,15,0.0

In [376]:
# test_null_6, test_6 = preproc_df(test_6)

20


In [486]:
a1, a2 = get_predict(booster, test_6), get_predict(booster_null, test_null_6)
ans = concat_data(a1, a2)
ans

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.537580,0.003562,0.018932,0.158147,0.004448,4.281308e-04,0.001052,0.000444,0.008151,0.012693,0.144100,0.000058,0.000682,9.004131e-07,0.004055,0.105634,0.000032
0,0.123687,0.215976,0.000230,0.005459,0.002249,3.203220e-05,0.001505,0.006826,0.001727,0.004468,0.039426,0.008190,0.005198,1.759006e-06,0.009166,0.137502,0.438358
1,0.085237,0.003624,0.002841,0.006903,0.025354,5.690028e-04,0.003537,0.017525,0.665914,0.002023,0.027264,0.000224,0.000752,1.428661e-05,0.006982,0.151225,0.000012
1,0.188023,0.376146,0.040802,0.000871,0.000736,5.158581e-04,0.009111,0.002979,0.004068,0.018320,0.036094,0.002029,0.010783,8.891837e-06,0.001874,0.305674,0.001967
2,0.026665,0.039437,0.005785,0.011253,0.002422,8.460631e-04,0.002838,0.000003,0.005014,0.067560,0.012301,0.000801,0.002503,1.962912e-06,0.000103,0.822459,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,0.315783,0.148770,0.035994,0.000656,0.000559,6.827186e-07,0.000076,0.000005,0.000277,0.000289,0.007982,0.055651,0.000046,6.529508e-06,0.002941,0.424578,0.006385
71605,0.025523,0.040883,0.009086,0.022167,0.005836,8.284300e-05,0.000166,0.000166,0.007652,0.002754,0.010486,0.006474,0.000183,2.389875e-05,0.004036,0.864453,0.000028
71606,0.070133,0.024635,0.022056,0.051218,0.043248,2.515969e-04,0.001415,0.000530,0.042212,0.002522,0.007118,0.000345,0.002874,1.703163e-06,0.011814,0.719598,0.000029
71607,0.012512,0.161281,0.004068,0.023647,0.003556,1.460871e-04,0.000181,0.000100,0.004830,0.002893,0.006051,0.000719,0.000556,2.697464e-06,0.002017,0.777273,0.000167


In [487]:
ans['id'] = list(test_df[test_df['date'] == 'month_6']['id'])
ans = ans[['id'] + list(ans.columns[:-1])]
ans

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.537580,0.003562,0.018932,0.158147,0.004448,4.281308e-04,0.001052,0.000444,0.008151,0.012693,0.144100,0.000058,0.000682,9.004131e-07,0.004055,0.105634,0.000032
0,200001,0.123687,0.215976,0.000230,0.005459,0.002249,3.203220e-05,0.001505,0.006826,0.001727,0.004468,0.039426,0.008190,0.005198,1.759006e-06,0.009166,0.137502,0.438358
1,200002,0.085237,0.003624,0.002841,0.006903,0.025354,5.690028e-04,0.003537,0.017525,0.665914,0.002023,0.027264,0.000224,0.000752,1.428661e-05,0.006982,0.151225,0.000012
1,200003,0.188023,0.376146,0.040802,0.000871,0.000736,5.158581e-04,0.009111,0.002979,0.004068,0.018320,0.036094,0.002029,0.010783,8.891837e-06,0.001874,0.305674,0.001967
2,200004,0.026665,0.039437,0.005785,0.011253,0.002422,8.460631e-04,0.002838,0.000003,0.005014,0.067560,0.012301,0.000801,0.002503,1.962912e-06,0.000103,0.822459,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28390,299995,0.315783,0.148770,0.035994,0.000656,0.000559,6.827186e-07,0.000076,0.000005,0.000277,0.000289,0.007982,0.055651,0.000046,6.529508e-06,0.002941,0.424578,0.006385
71605,299996,0.025523,0.040883,0.009086,0.022167,0.005836,8.284300e-05,0.000166,0.000166,0.007652,0.002754,0.010486,0.006474,0.000183,2.389875e-05,0.004036,0.864453,0.000028
71606,299997,0.070133,0.024635,0.022056,0.051218,0.043248,2.515969e-04,0.001415,0.000530,0.042212,0.002522,0.007118,0.000345,0.002874,1.703163e-06,0.011814,0.719598,0.000029
71607,299998,0.012512,0.161281,0.004068,0.023647,0.003556,1.460871e-04,0.000181,0.000100,0.004830,0.002893,0.006051,0.000719,0.000556,2.697464e-06,0.002017,0.777273,0.000167


In [488]:
ans.to_csv("output_20.csv", index=False)

In [88]:
ans.to_csv("mega_giga4.csv", index=False)

In [489]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'output_20.csv')

/kaggle/working/output_20.csv

<a href="/kaggle/working/mega_giga4.csv"> Download File </a>

In [89]:
pd.read_csv('/kaggle/working/mega_giga4.csv')

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.021474,0.014307,0.007679,0.032879,0.036636,0.000039,0.002153,1.989868e-04,0.005683,0.014941,0.018640,0.000975,0.000149,3.903690e-06,0.052337,0.791766,1.399792e-04
1,200001,0.028894,0.785682,0.000833,0.007961,0.000128,0.000040,0.000240,1.882026e-04,0.004002,0.003199,0.003057,0.000102,0.003225,2.426525e-06,0.002235,0.160173,3.880411e-05
2,200002,0.893385,0.034218,0.000525,0.032847,0.002959,0.000208,0.000666,4.593753e-04,0.009727,0.000593,0.006337,0.000497,0.000014,3.099485e-06,0.001459,0.016081,2.029733e-05
3,200003,0.044534,0.737292,0.000087,0.001004,0.000095,0.000379,0.000034,9.210503e-05,0.000416,0.033728,0.001506,0.000014,0.000113,1.144832e-05,0.002553,0.178141,8.989123e-07
4,200004,0.244222,0.116345,0.027090,0.052287,0.001140,0.001532,0.001644,2.674975e-05,0.001594,0.233491,0.016897,0.000258,0.131015,1.397883e-05,0.002421,0.168960,1.061813e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.015437,0.412114,0.001055,0.002441,0.001542,0.000003,0.000061,8.579939e-07,0.001245,0.000380,0.001397,0.000017,0.000041,7.250012e-05,0.000423,0.563759,1.083279e-05
99996,299996,0.008409,0.021737,0.006059,0.009271,0.000570,0.000077,0.000128,4.419316e-06,0.001562,0.002400,0.001742,0.002147,0.000357,6.619391e-06,0.000314,0.945202,1.405203e-05
99997,299997,0.030102,0.012670,0.034435,0.072596,0.010702,0.000095,0.000023,2.157205e-05,0.055226,0.003071,0.001656,0.000031,0.000277,5.309905e-07,0.015680,0.763413,1.341418e-06
99998,299998,0.054055,0.034581,0.007635,0.021081,0.004010,0.000754,0.000131,2.237021e-03,0.012449,0.014501,0.041967,0.053888,0.000138,5.924435e-06,0.601486,0.132905,1.817545e-02
